In [4]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

from scipy.stats import binom
from scipy.stats import poisson
from scipy.stats import chi2

In [5]:
df = pd.read_csv('../Data/Sesion7/Binomial.csv')

In [6]:
df

,NCaras,ObsFreq
0,0,38
1,1,143
2,2,343
3,3,285
4,4,166
5,5,25
